In [1]:
from flowprintOptimal.sekigo.train import Trainer
import warnings
warnings.filterwarnings('ignore')
from flowprintOptimal.sekigo.utils.evaluations import EarlyEvaluation
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from flowprintOptimal.sekigo.utils.documentor import Documenter

In [2]:
configs = dict(
    name = "deployment_fs",
    description = "deployment_fs",
    
    common_config = dict(
        max_timesteps = 15,
        min_timesteps = 8
    ),
    
    full_model_kwargs = dict(
        lstm_hidden_size = 256,
        layers= 2, lstm_input_size = 3
    ),

    early_model_kwargs = dict(
        lstm_input_size= 3,lstm_hidden_size= 256,layers = 2                    
    ),
    
    data_config = dict(
        dataset_name = "deployment_fs",
        subsampleConfig = None,#dict(max_gap = 20, min_gap = 5),                             
        max_flow_length = 100, # in seconds  ( each flow sample cannot excede this length)
        test_size = .2,
        ood_classes = [],
        do_balance = False,
        data_type = "packet_representation",
        truncate_length = 20

    ),

    rewarder_config = dict(
        l = .5
    ),

    dataset_config = dict(
        aug = [0,.2]
    ),

    memory_fillter_config = dict(
        ood_config = dict(ood_aug = [.6,.9], ood_prob = .2),
        min_length = 1,
        use_balancer = False
    ),
    full_trainer_config = dict(
        use_sampler = False
    ),
    early_trainer_config = dict(
        use_sampler = False  # this is for giving more weight to wait samples
    )

)

In [3]:
trainer = Trainer(configs = configs)

In [4]:
trainer.start()

start
data/ClassificationOnlyFlows/deployment_fs
Got datasets
534720
 1    132720
 3    117840
 2    111480
 0     88800
-1     53280
 4     30600
Name: count, dtype: int64
 ---- 1000 metric q_loss = 0.4854807216823101
updated best with f1 = 0.23777845590479094
 ---- 1 metric test_eval_f1 = 0.23777845590479094
 ---- 1 metric test_eval_time = 14.705521472392638
 ---- 1 metric incorrect_ood_test = 0.7073608617594255
 ---- 2000 metric q_loss = 0.45444431874156
updated best with f1 = 0.34494598748245986
 ---- 2 metric test_eval_f1 = 0.34494598748245986
 ---- 2 metric test_eval_time = 8.067469879518072
 ---- 2 metric incorrect_ood_test = 0.25493716337522443
 ---- 1 metric train_eval_f1 = 0.35505072606237337
 ---- 1 metric train_eval_time = 8.647165259348613
 ---- 1 metric incorrect_ood_train = 0.25583482944344704
 ---- 3000 metric q_loss = 0.42529991719126703
updated best with f1 = 0.5485506919448013
 ---- 3 metric test_eval_f1 = 0.5485506919448013
 ---- 3 metric test_eval_time = 7.52972972

KeyboardInterrupt: 

In [5]:
trainer.ddq_model.best["score"]

0.9218077449722977

In [ ]:
documentor = Documenter(train_dataset= trainer.train_dataset,test_dataset= trainer.test_dataset, ood_dataset = trainer.ood_dataset,full_model= trainer.ddq_model.best["model"], early_model= trainer.ddq_model.best["model"],
                        configs= configs
                        )
documentor.document(name= configs["name"])

In [10]:
documentor = Documenter.load("deployment_sm")

Loading dummy full model


In [11]:
test_dataset = documentor.test_dataset
len(test_dataset)

416

In [12]:
eval = EarlyEvaluation(min_steps= 1,device= device,model= documentor.early_model).getMetrices(dataset= documentor.test_dataset,ood_dataset= documentor.ood_dataset)

In [13]:
eval["cm"]

array([[ 13,   4,   0,   1,   0,   9],
       [  4,  38,   3,   4,   1,   5],
       [  0,   3,  38,   0,   0,   3],
       [  5,   1,   3, 147,  13,  10],
       [  0,   0,   0,  25,  24,   0],
       [ 10,   5,   3,   1,   0,  42]])

In [14]:
eval

{'micro_f1': 0.727710843373494,
 'macro_f1': 0.6690385548652649,
 'accuracy': 0.727710843373494,
 'cm': array([[ 13,   4,   0,   1,   0,   9],
        [  4,  38,   3,   4,   1,   5],
        [  0,   3,  38,   0,   0,   3],
        [  5,   1,   3, 147,  13,  10],
        [  0,   0,   0,  25,  24,   0],
        [ 10,   5,   3,   1,   0,  42]]),
 'per_class_f1': array([0.44067797, 0.71698113, 0.83516484, 0.82352941, 0.55172414,
        0.64615385]),
 'time': 9.602409638554217,
 'time_std': 5.129033256418724,
 'incorrect_ood': 0.002403846153846154}

In [15]:
matrix = eval["cm"]
print(documentor.test_dataset.label_to_index)
matrix.diagonal()/matrix.sum(axis=1)

{'Twitter': 0, 'LinkedIn': 1, 'TikTok': 2, 'Facebook': 3, 'Instagram': 4, 'Reddit': 5}


array([0.48148148, 0.69090909, 0.86363636, 0.82122905, 0.48979592,
       0.68852459])

In [17]:
eval

{'micro_f1': 0.727710843373494,
 'macro_f1': 0.6690385548652649,
 'accuracy': 0.727710843373494,
 'cm': array([[ 13,   4,   0,   1,   0,   9],
        [  4,  38,   3,   4,   1,   5],
        [  0,   3,  38,   0,   0,   3],
        [  5,   1,   3, 147,  13,  10],
        [  0,   0,   0,  25,  24,   0],
        [ 10,   5,   3,   1,   0,  42]]),
 'per_class_f1': array([0.44067797, 0.71698113, 0.83516484, 0.82352941, 0.55172414,
        0.64615385]),
 'time': 9.602409638554217,
 'time_std': 5.129033256418724,
 'incorrect_ood': 0.002403846153846154}

In [18]:
pred,packet_taken,labels = EarlyEvaluation(min_steps= 1,device= device,model= documentor.early_model).predictOnDataset(dataset= documentor.test_dataset)

In [19]:
label_to_index = documentor.test_dataset.label_to_index

In [20]:
for name,index in label_to_index.items():
    fpr = ((pred == -1)&(labels == index)).sum()/((labels == index).sum())
    print(name, fpr)

Twitter 0.0
LinkedIn 0.0
TikTok 0.0
Facebook 0.005555555555555556
Instagram 0.0
Reddit 0.0


In [21]:
for name,index in label_to_index.items():
    t_label = packet_taken[labels == index]
    print(name, t_label.mean(), t_label.std())

Twitter 9.74074074074074 4.427126754810055
LinkedIn 10.527272727272727 5.263440096827603
TikTok 6.318181818181818 3.521187346702602
Facebook 9.905555555555555 5.23205634337465
Instagram 11.16326530612245 5.5745300480079285
Reddit 9.098360655737705 4.622485734075414


In [22]:
import numpy as np
from flowprintOptimal.sekigo.flowUtils.commons import dropPacketFromPacketRep, getTimeStampsFromIAT
from joblib import Parallel, delayed

In [23]:
dataset = documentor.test_dataset

In [24]:
_,packet_taken,_ = EarlyEvaluation(min_steps= 1,device= device,model= documentor.early_model).predictOnDataset(dataset= dataset)

In [25]:
IATs = [p.inter_arrival_times for p in dataset.flows]
timestamps = Parallel(n_jobs=10)(delayed(getTimeStampsFromIAT)(iat) for iat in IATs)

In [26]:
def convetPacketsTakenToTimeTaken(timestamps,packets_taken):
    time_taken = []
    for i in range(len(packets_taken)):
        time_taken.append((timestamps[i][packets_taken[i] - 1] - timestamps[i][0]).total_seconds() )
    
    return np.array(time_taken)


time_taken = convetPacketsTakenToTimeTaken(timestamps= timestamps, packets_taken= packet_taken)

In [27]:
time_taken.mean(), time_taken.std()

(0.4023385432692308, 3.243947830354812)

In [28]:
for name,index in label_to_index.items():
    t_label = time_taken[labels == index]
    print(name, t_label.mean(), t_label.std())

Twitter 2.948781851851852 10.342726184024174
LinkedIn 0.27158380000000004 1.6295024286113442
TikTok 0.13042127272727272 0.18921756672933696
Facebook 0.06348779444444445 0.28307474590012244
Instagram 0.29100461224489793 1.3902223097047497
Reddit 0.6785745901639345 4.1041018335218915
